## Generative Adversarial Network (GAN) Image Generation

In [ ]:
# implement GAN generator network
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np

latent_dim = 32
height, width = 32, 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# transform input into 16x16 128-channel feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# upsample to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# output is shape (32, 32, 3)
generator_output = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, generator_output)
generator.summary()

In [ ]:
# implement GAN discriminator network

discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x) # trick which adds stochasticity

discriminator_output = layers.Dense(1, activation='sigmoid')(x)  # binary classification

discriminator = keras.models.Model(discriminator_input, discriminator_output)
discriminator.summary()

discriminator_optimizer = keras.optimizers.legacy.RMSprop(
                                learning_rate=0.0008, clipvalue=1.0, decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [10]:
# implement deep convolutional GAN (DCGAN) model

discriminator.trainable = False # discriminator is frozen while training generator

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.legacy.RMSprop(
    learning_rate=0.0004, clipvalue=1.0, decay=1e-8)

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [ ]:
# train GAN
import os
from tensorflow.keras.preprocessing import image

iterations = 1000
batch_size = 20
save_dir = './gan_output/'

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 6] # only frog images
x_train = x_train.reshape((x_train.shape[0],) +
                          (height, width, channels)).astype('float32') / 255.0

start = 0
for step in range(iterations):
    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # use points to generate (decode into) images
    generated_images = generator.predict(random_latent_vectors)
    
    # combine generated images with real images
    real_images = x_train[start:start + batch_size]
    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape) # trick to add random noise
    
    # train discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # sample new random latent points
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    misleading_targets = np.zeros((batch_size, 1)) # labels fake images as real
    
    # train generator (via gan)
    g_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    # loop back to start of training data if needed
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    # save progress every 100 steps
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print('step:', step)
        print('discriminator loss:', d_loss)
        print('generator loss:', g_loss)
        
        img = image.array_to_img(generated_images[0] * 255.0, scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255.0, scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))